<a href="https://colab.research.google.com/github/kazuki-komori/coursera/blob/main/regression/CSR_regression_w1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 第1週 勉強会

## Fitting a simple linear regression model on housing data
### 住宅データに対する単回帰によるモデルのあてはめ

[資料はこちら](https://www.coursera.org/learn/ml-regression/supplement/z0Uef/fitting-a-simple-linear-regression-model-on-housing-data)

今回取り組む内容は以下の通り

- SArray関数とSFrame関数を使用して、重要な統計情報を計算する
- 閉じた形式の解を使って，単回帰重みを計算する関数を作成する
- 入力の特徴量が与えられたときに，出力の予測を行う関数を書く
- 回帰を逆にして，出力が与えられたときの入力/特徴を予測する
- 住宅価格を予測するための2つの異なるモデルを比較する

## setup

In [4]:
!unzip -q /content/drive/MyDrive/coursera/home_data.zip

In [ ]:
!pip install turicreate

In [9]:
import turicreate
import numpy as np

## データの読み込み
(1) SFrame 形式でデータを読み込んでみる

In [3]:
df_house = turicreate.SFrame('/content/home_data.sframe')
df_house.head(5)

In [5]:
df_house.head(5)

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat
0,3,7.0,1180.0,0.0,1955.0,0.0,98178,47.51123398
0,3,7.0,2170.0,400.0,1951.0,1991.0,98125,47.72102274
0,3,6.0,770.0,0.0,1933.0,0.0,98028,47.73792661
0,5,7.0,1050.0,910.0,1965.0,0.0,98136,47.52082


## データの分割
(2) データを80%のトレーニングデータと20%のテストデータに分割する

In [7]:
df_train, df_test = df_house.random_split(.8, seed=0)

## 単回帰の実装

単回帰の式を以下のように定義する.

$$y^{\prime} = w_0^{\prime} + w_1^{\prime} \cdot x$$

これより, 偏微分を行うと回帰係数は以下のように導出される. (refs: Good Note)

$$w_0^{\prime} = \bar{y} - w_1^{\prime} \cdot \bar{x}$$

$$w_1^{\prime} = \frac{S_{xy}}{S_{x}^{2}}$$


In [27]:
## (3)
def simple_linear_regression(input_feature, output):
  w_1 = np.cov(input_feature, output)[0][1] / (input_feature.var())

  x_bar = input_feature.mean()
  y_bar = output.mean()
  w_0 =  y_bar - w_1 * x_bar
  return w_0, w_1

## (4) `sqft_living` を説明変数, `price` を目的変数として計算してみる


In [29]:
intercept, slope = simple_linear_regression(df_train["sqft_living"], df_train["price"])
print(f"切片: {intercept}, 傾き: {slope}")

切片: -47149.81544077501, 傾き: 281.97505894614585


## (5) intercept と slope を用いた推定用の関数を定義

上記の回帰式を定義

In [30]:
def get_regression_predictions(input_feature, intercept, slope):
  predicted_output = intercept + input_feature * slope
  return predicted_output

## (6) 実際に `sqft = 2650` として推定してみる

In [32]:
get_regression_predictions(2650, intercept, slope)

700084.0907665115

## (7) Residual Sum of Squares (残差二乗和)

残差の式は以下のように定義される.

$$RSS = \sum_{i=1}^{n}(y_i - y_i^{\prime})^2$$

In [53]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
  y_dash = get_regression_predictions(input_feature, intercept, slope)
  y = output.to_numpy()
  RSS = (y - y_dash).sum()
  return RSS

(8) 訓練データに対して RSS を計算してみる

In [64]:
get_residual_sum_of_squares(df_train["sqft_living"], df_train["price"], intercept, slope)

1.0542571544647217e-05

## (9) 単回帰は切片と傾きをそのままで説明変数と目的変数を入れ替えることができる

$$y^{\prime} = w_0^{\prime} + w_1^{\prime} \cdot x$$

逆関数は以下の通り

$$x^{\prime} = \frac{1}{w_1}(y^{\prime} - w_0)$$

In [68]:
def inverse_regression_predictions(output, intercept, slope):
  estimated_input = (output - intercept) / slope
  return estimated_input

## (10) 実際の家賃が $800,000 のとき, リビングの広さを推定してみる

In [69]:
inverse_regression_predictions(800000, intercept, slope)

3004.343074195691

## `sqft_living` の代わりに `bedrooms` を使用して分析する

In [70]:
intercept_bedrooms, slope_bedrooms = simple_linear_regression(df_train["bedrooms"], df_train["price"])

## (12) 2つの RSS を比較する

In [73]:
RSS_sqft = get_residual_sum_of_squares(df_train["sqft_living"], df_train["price"], intercept, slope)
print(f"リビングのRSS: {RSS_sqft}")
RSS_bedrooms = get_residual_sum_of_squares(df_train["bedrooms"], df_train["price"], intercept_bedrooms, slope_bedrooms)
print(f"ベッドルームのRSS: {RSS_bedrooms}")

リビングのRSS: 1.0542571544647217e-05
ベッドルームのRSS: 3.492087125778198e-05


## (13) 比較した結果

`bedrooms` の RSS の方が大きいため, `sqft_living` の方が当てはまりが良い

In [74]:
RSS_sqft < RSS_bedrooms

True